In [1]:
import pandas as pd

In [3]:
messages = pd.read_csv('/home/gourav/Documents/nlp/smsspamcollection/SMSSpamCollection',sep='\t',
                      names=['label','message'])

In [5]:
import re
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gourav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [9]:
ps = PorterStemmer()
corpus = []

for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [10]:
#Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [11]:
y = pd.get_dummies(messages['label'])
y = y.iloc[:,1].values

In [12]:
#Train - test Split
from sklearn.model_selection import train_test_split
X_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=0)

In [13]:
# Training Naive Bayes classifier
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [14]:
y_pred = spam_detect_model.predict(x_test)

In [16]:
#confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test, y_pred=y_pred)

In [17]:
confusion_m

array([[946,   9],
       [  7, 153]])

In [27]:
# Accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)

In [28]:
# accuracy = accuracy*100
accuracy = accuracy*100

In [29]:
accuracy

98.56502242152466